In [1]:
import mediapipe as mp
import cv2
import numpy as np
import socket

host, port = "10.10.173.35", 25467

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((host, port))

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        results = hands.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)
                image_height,image_width,_ =image.shape
                for hand_landmarks in results.multi_hand_landmarks:
                    y=hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y*image_height
                    mapped = 5 + (y - 100) * -1 / 40;
                    data=str(mapped)
                    sock.sendall(data.encode())
                    

        cv2.imshow('Hand Tracking', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()